Copyright (c) 2019 [윤기태]

https://github.com/yoonkt200/python-data-analysis

[MIT License](https://github.com/yoonkt200/python-data-analysis/blob/master/LICENSE.txt)

# (가제) 파이썬 데이터 분석

-----

# 웹 페이지 크롤링

### 1. 대상 페이지의 구조 살펴보기

- Internet Explorer, Chrome 등의 웹 브라우저로 크롤링의 대상이 될 페이지 구조 살펴보기
    - 웹 브라우저의 개발자 도구 실행 (Chrome 기준 설명)
        - `Chrome 맞춤설정 및 제어 탭 - 도구 더보기 - 개발자 도구` 클릭
        - 혹은 `Command + Alt + I (Mac), Ctrl + Shift + I (윈도우)`

![앱 생성 페이지](img/crawl_1.png)

-----

### 2. 페이지들의 URL 정보 추출

- 개발자 도구를 통해, 텍스트 데이터를 가져올 페이지들의 URL 리스트 추출
    - HTML 구조에서 링크 정보 (**a 태그**) 추출
        - 개발자도구 좌측 상단의 `마우스 모양` 버튼 클릭
        - HTML 구조를 보고싶은 영역으로 마우스 커서 이동

![앱 생성 페이지](img/crawl_2.png)

-----

- 아래 코드 실행을 위해, anaconda prompt 혹은 Terminal에서 아래와 같은 패키지들을 설치해 줍니다.
    - (env_name) `pip install selenium beautifulsoup4`
- 혹은 아래의 코드로 라이브러리를 설치합니다.

In [1]:
!pip install selenium beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 16.0 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


-----

In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re

# brew 로 설치된 chromedriver의 path (Mac)
# 실행 권한 오류가 생길 시, chromedriver가 있는 폴더로 이동하여 `xattr -d com.apple.quarantine chromedriver` 명령어 입력
mac_path = "/usr/local/bin/chromedriver"  # m1 이전 맥 OS
# mac_path = "/opt/homebrew/bin/chromedriver"  # m1 이후 맥 OS

# 윈도우용 크롬 웹드라이버 실행 경로 (Windows)
window_path = "chromedriver.exe"

# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://namu.wiki/RecentChanges"

# 크롬 드라이버를 사용합니다 (맥은 첫 줄, 윈도우는 두번째 줄 실행)
driver = webdriver.Chrome(mac_path)

# 드라이버가 브라우징 할 페이지 소스를 입력합니다
driver.get(source_url)
req = driver.page_source

# 사이트의 html 구조에 기반하여 데이터를 파싱합니다.
soup=BeautifulSoup(req, "html.parser")
contents_table = soup.find(name="table")
table_body = contents_table.find(name="tbody")
table_rows = table_body.find_all(name="tr")

# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
page_url_base = "https://namu.wiki"
page_urls = []
for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all("td")[0]
    td_url = first_td.find_all("a")
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get("href")
        if "png" not in page_url:
            page_urls.append(page_url)

# 중복 url을 제거합니다.
page_urls = list(set(page_urls))
for page in page_urls[:3]:
    print(page)
    
# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

/var/folders/ww/prhhsq7100g92nt7xr4ltql5lkv223/T/ipykernel_30754/3517989882.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(mac_path)


https://namu.wiki/w/%ED%99%8B%EC%BD%94%20%ED%83%80%EB%A3%A8%EB%A7%88%EC%97%90(%EC%9A%B0%EB%A7%88%EB%AC%B4%EC%8A%A4%EB%A9%94%20%ED%94%84%EB%A6%AC%ED%8B%B0%20%EB%8D%94%EB%B9%84)
https://namu.wiki/w/%EB%A1%9C%EB%B2%84%ED%8A%B8%20%EB%93%80%EB%B0%9C
https://namu.wiki/w/%EB%82%A8%EC%9E%90%EA%B0%80%20%EC%82%AC%EB%9E%91%ED%95%A0%20%EB%95%8C(SBS%20%EB%93%9C%EB%9D%BC%EB%A7%88)


- 위의 코드를 통해 https://namu.wiki/RecentChanges 페이지의 최근 변경 문서 링크들을 추출.

-----

### 3. 각종 텍스트 정보 추출

- 동일한 방법으로, 개발자 도구를 이용하여 최근 변경된 나무위키 페이지의 텍스트 정보 추출
    - `문서 제목`
    - `문서 카테고리`
    - `문서 내용`

![앱 생성 페이지](img/crawl_3.png)

![앱 생성 페이지](img/crawl_4.png)

![앱 생성 페이지](img/crawl_5.png)

- `2. 페이지들의 URL 정보 추출` 에서 추출했던 URL 페이지 정보를 기반으로 크롤링 실행.

In [8]:
# 하나의 최근 변경된 문서를 크롤링합니다.

# 크롬 드라이버를 사용합니다 (맥은 첫 줄, 윈도우는 두번째 줄 실행)
driver = webdriver.Chrome(mac_path)
# driver = webdriver.Chrome(window_path)

# 드라이버가 브라우징 할 페이지 소스를 입력합니다
driver.get(page_urls[0])
req = driver.page_source
soup=BeautifulSoup(req, "html.parser")
contents_table = soup.find(name="article")
title = contents_table.find_all("h1")[0]
category = contents_table.find_all("ul")[0]
content_paragraphs = contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})
content_corpus_list = []

for paragraphs in content_paragraphs:
    content_corpus_list.append(paragraphs.text)
content_corpus = "".join(content_corpus_list)

print(title.text)
print("\n")
print(category.text)
print("\n")
print(content_corpus)

# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()